In [2]:
import pandas as pd
import lightgbm as lgb
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv('train_feature_v1.csv')
test = pd.read_csv('test_feature_b_v1.csv')

In [4]:
train_xy, offline_test = train_test_split(train, test_size = 0.2)
trainset,val = train_test_split(train_xy, test_size = 0.2)
y = trainset.label
X = trainset.drop(['uid','label'],axis=1)
val_y = val.label
val_X = val.drop(['uid','label'],axis=1)  
offline_test_X = offline_test.drop(['uid','label'],axis=1)
online_test_X = test.drop(['uid'],axis=1)
lgb_train = lgb.Dataset(X, y, free_raw_data=False)
lgb_eval = lgb.Dataset(val_X, val_y, reference=lgb_train, free_raw_data=False)

In [5]:
train_set = lgb.Dataset(train.drop(['uid','label'],axis=1),label=train.label)
#test_set = lgb.Dataset(test.drop(['uid'],axis=1))

In [6]:
def evalMetric(preds,train):
    label = train.label
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre = pre.sort_values(by='preds',ascending=False)
    auc = metrics.roc_auc_score(pre.label,pre.preds)
    pre.preds = pre.preds.map(lambda x:1 if x>=0.5 else 0)
    f1 = metrics.f1_score(pre.label,pre.preds)
    res = 0.6*auc + 0.4*f1
    return 'res',res,True

In [7]:
lgb_params = {
              'boosting_type': 'gbdt',
              'objective': 'binary',
            'is_training_metric': False,

              'learning_rate': 0.08,
              'num_leaves':16,
              'max_depth':6,
  
              'min_data_in_leaf':40,
               'verbosity':-1,
                'is_training_metric':False,
    
             'feature_fraction': 0.8,
              'bagging_fraction': 0.8,
              'bagging_freq':1,
              'is_unbalance':True
}

In [8]:
'''
print("调参1：提高准确率")
for num_leaves in range(20,200,5):
    for max_depth in range(3,8,1):
        params['num_leaves'] = num_leaves
        params['max_depth'] = max_depth
        cv_results = lgb.cv(
                            params,
                            lgb_train,
                            seed=2018,
                            nfold=3,
                            metrics=['evalMetric'],
                            early_stopping_rounds=10,
                            verbose_eval=True,
                            feval=evalMetric
                            )
        mean_merror = pd.Series(cv_results['res-mean']).min()
        boost_rounds = pd.Series(cv_results['res-mean']).argmin()
    
        if mean_merror < min_merror:
            min_merror = mean_merror
            best_params['num_leaves'] = num_leaves
            best_params['max_depth'] = max_depth
        
params['num_leaves'] = best_params['num_leaves']
params['max_depth'] = best_params['max_depth']
'''


'\nprint("调参1：提高准确率")\nfor num_leaves in range(20,200,5):\n    for max_depth in range(3,8,1):\n        params[\'num_leaves\'] = num_leaves\n        params[\'max_depth\'] = max_depth\n        cv_results = lgb.cv(\n                            params,\n                            lgb_train,\n                            seed=2018,\n                            nfold=3,\n                            metrics=[\'evalMetric\'],\n                            early_stopping_rounds=10,\n                            verbose_eval=True,\n                            feval=evalMetric\n                            )\n        mean_merror = pd.Series(cv_results[\'res-mean\']).min()\n        boost_rounds = pd.Series(cv_results[\'res-mean\']).argmin()\n    \n        if mean_merror < min_merror:\n            min_merror = mean_merror\n            best_params[\'num_leaves\'] = num_leaves\n            best_params[\'max_depth\'] = max_depth\n        \nparams[\'num_leaves\'] = best_params[\'num_leaves\']\nparams[\'ma

In [9]:
'''
print("调参2：降低过拟合")

for max_bin in range(1,255,5):
    for min_data_in_leaf in range(10,200,5):
            params['max_bin'] = max_bin
            params['min_data_in_leaf'] = min_data_in_leaf

            cv_results = lgb.cv(
                                params,
                                lgb_train,
                                seed=42,
                                nfold=3,
                                metrics=['evalMetric'],
                                early_stopping_rounds=3,
                                verbose_eval=True
                                )
                
            mean_merror = pd.Series(cv_results['binary_error-mean']).min()
            boost_rounds = pd.Series(cv_results['binary_error-mean']).argmin()
            if mean_merror < min_merror:
                min_merror = mean_merror
                best_params['max_bin']= max_bin
                best_params['min_data_in_leaf'] = min_data_in_leaf
params['min_data_in_leaf'] = best_params['min_data_in_leaf']
params['max_bin'] = best_params['max_bin']
'''

'\nprint("调参2：降低过拟合")\n\nfor max_bin in range(1,255,5):\n    for min_data_in_leaf in range(10,200,5):\n            params[\'max_bin\'] = max_bin\n            params[\'min_data_in_leaf\'] = min_data_in_leaf\n\n            cv_results = lgb.cv(\n                                params,\n                                lgb_train,\n                                seed=42,\n                                nfold=3,\n                                metrics=[\'evalMetric\'],\n                                early_stopping_rounds=3,\n                                verbose_eval=True\n                                )\n                \n            mean_merror = pd.Series(cv_results[\'binary_error-mean\']).min()\n            boost_rounds = pd.Series(cv_results[\'binary_error-mean\']).argmin()\n            if mean_merror < min_merror:\n                min_merror = mean_merror\n                best_params[\'max_bin\']= max_bin\n                best_params[\'min_data_in_leaf\'] = min_data_in_leaf\npa

In [10]:
'''
print("调参3：降低过拟合")
for feature_fraction in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    for bagging_fraction in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        for bagging_freq in range(0,50,5):
            params['feature_fraction'] = feature_fraction
            params['bagging_fraction'] = bagging_fraction
            params['bagging_freq'] = bagging_freq
            cv_results = lgb.cv(
                                params,
                                lgb_train,
                                seed=42,
                                nfold=3,
                                metrics=['evalMetric'],
                                early_stopping_rounds=3,
                                verbose_eval=True
                                )
                
            mean_merror = pd.Series(cv_results['binary_error-mean']).min()
            boost_rounds = pd.Series(cv_results['binary_error-mean']).argmin()

            if mean_merror < min_merror:
                min_merror = mean_merror
                best_params['feature_fraction'] = feature_fraction
                best_params['bagging_fraction'] = bagging_fraction
                best_params['bagging_freq'] = bagging_freq
params['feature_fraction'] = best_params['feature_fraction']
params['bagging_fraction'] = best_params['bagging_fraction']
params['bagging_freq'] = best_params['bagging_freq']
'''

'\nprint("调参3：降低过拟合")\nfor feature_fraction in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:\n    for bagging_fraction in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:\n        for bagging_freq in range(0,50,5):\n            params[\'feature_fraction\'] = feature_fraction\n            params[\'bagging_fraction\'] = bagging_fraction\n            params[\'bagging_freq\'] = bagging_freq\n            cv_results = lgb.cv(\n                                params,\n                                lgb_train,\n                                seed=42,\n                                nfold=3,\n                                metrics=[\'evalMetric\'],\n                                early_stopping_rounds=3,\n                                verbose_eval=True\n                                )\n                \n            mean_merror = pd.Series(cv_results[\'binary_error-mean\']).min()\n            boost_rounds = pd.Series(cv_results[\'binary_error-mean\']).argmin()\n\n            if mean_merr

In [11]:
#lgb.cv(lgb_params,lgb_train,feval=evalMetric,early_stopping_rounds=100,verbose_eval=5,num_boost_round=10000,nfold=3,metrics=['evalMetric'])
#model =lgb.train(lgb_params,lgb_train,feval=evalMetric,verbose_eval=5,num_boost_round=300,valid_sets=lgb_eval)
bst = lgb.cv(lgb_params,train_set,feval=evalMetric,early_stopping_rounds=100,verbose_eval=5,num_boost_round=10000,nfold=3,metrics=['evalMetric'])
model =lgb.train(lgb_params,train_set,feval=evalMetric,verbose_eval=5,early_stopping_rounds=100,num_boost_round=385,valid_sets=train_set)

print('线下测试:')
preds_offline = model.predict(offline_test_X,num_iteration=model.best_iteration) # 输出概率
offline=offline_test[['uid','label']]
offline['preds']=preds_offline
print(evalMetric(offline.preds, offline))

preds_online = model.predict(online_test_X,num_iteration=model.best_iteration)  # 输出概率

res =pd.DataFrame({'uid':test.uid,'label':preds_online})
res=res.sort_values(by='label',ascending=False)
res.label=res.label.map(lambda x: 1 if x>=0.5 else 0)
res.label = res.label.map(lambda x: int(x))
res.to_csv('lgb-baseline_B_v3.csv',index=False,header=False,sep=',',columns=['uid','label'])

df = pd.DataFrame(offline_test_X.columns.tolist(), columns=['feature'])
df['importance']=list(model.feature_importance())
df = df.sort_values(by='importance',ascending=False)
df.to_csv("feature_score.csv",index=None,encoding='gbk')

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[5]	cv_agg's res: 0.561588 + 0.00235808


[10]	cv_agg's res: 0.838501 + 0.0101515


[15]	cv_agg's res: 0.853793 + 0.00709265


[20]	cv_agg's res: 0.860298 + 0.00924799


...

[360]	training's res: 1
[365]	training's res: 1


[370]	training's res: 1
[375]	training's res: 1


[380]	training's res: 1
Early stopping, best iteration is:
[283]	training's res: 1
线下测试:


('res', 1.0, True)


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
df

,feature,importance
19,sms_opp_head_max_in,92
61,voice_len_mean_in,70
15,sms_opp_len_mean_in,57
110,voice_cnt_peruniquecall,56
170,wa_type1_date_mean,54
193,wa_dura_sum_percnt,50
194,wa_type0_namecnt_perday,50
21,sms_opp_head_min_in,49
69,voice_head_mean_in,47
17,sms_opp_head_mean_in,41


In [13]:
# train_df = pd.read_csv('uid_train.txt',sep='\t',header=None)
# train_df.columns = ['uid','label']
# dtrain_t = train_df
# test_df = pd.read_csv('uid_test_b_v2.txt',sep='\t',header=None)
# test_df.columns = ['uid']
# dtest_t = test_df
# dtrain_t = train.drop(['uid','label'],axis=1)
# dtrain_t = dtrain_t.drop(df[df['importance']<100]['feature'],axis=1)
# dtest_t = test.drop(['uid'],axis=1)
# dtest_t = dtest_t.drop(df[df['importance']<100]['feature'],axis=1)
# dtrain_t.to_csv('train_feature_200.csv',index=None)
# dtest_t.to_csv('test_feature_b200.csv',index=None)